라이브러리 설치

In [ ]:
#링크 수집을 위한 라이브러리 설치
!pip install selenium

라이브러리 임포트

In [ ]:
from bs4 import BeautifulSoup
import requests
from selenium import webdriver as wd
from selenium.webdriver.common.keys import Keys
import time
import re
import pandas as pd

In [ ]:
# install chromium, its driver, and selenium
!apt-get update
!apt install chromium-chromedriver
!cp /usr/lib/chromium-browser/chromedriver /usr/bin
!pip install selenium
# set options to be headless, ..
from selenium import webdriver
options = webdriver.ChromeOptions()
options.add_argument('--headless')
options.add_argument('--no-sandbox')
options.add_argument('--disable-dev-shm-usage')

driver = webdriver.Chrome('chromedriver',options=options)


Hit:1 https://cloud.r-project.org/bin/linux/ubuntu bionic-cran40/ InRelease
Ign:2 https://developer.download.nvidia.com/compute/cuda/repos/ubuntu1804/x86_64  InRelease
Ign:3 https://developer.download.nvidia.com/compute/machine-learning/repos/ubuntu1804/x86_64  InRelease
Hit:4 https://developer.download.nvidia.com/compute/cuda/repos/ubuntu1804/x86_64  Release
Hit:5 https://developer.download.nvidia.com/compute/machine-learning/repos/ubuntu1804/x86_64  Release
Hit:6 http://ppa.launchpad.net/c2d4u.team/c2d4u4.0+/ubuntu bionic InRelease
Get:7 http://security.ubuntu.com/ubuntu bionic-security InRelease [88.7 kB]
Hit:8 http://archive.ubuntu.com/ubuntu bionic InRelease
Get:10 http://archive.ubuntu.com/ubuntu bionic-updates InRelease [88.7 kB]
Hit:11 http://ppa.launchpad.net/cran/libgit2/ubuntu bionic InRelease
Hit:13 http://ppa.launchpad.net/deadsnakes/ppa/ubuntu bionic InRelease
Get:14 http://archive.ubuntu.com/ubuntu bionic-backports InRelease [74.6 kB]
Hit:15 http://ppa.launchpad.net/grap

분석할 데이터 모으기 (크롤링)

네이버 영화 TOP100 페이지 별 링크 수집

In [ ]:
#https://serieson.naver.com/movie/top100List.nhn?&rankingTypeCode=PC_R&page= 실시간
#https://serieson.naver.com/movie/top100List.nhn?&rankingTypeCode=PC_D&page= 일간
#https://serieson.naver.com/movie/top100List.nhn?&rankingTypeCode=PC_W&page= 주간
#https://serieson.naver.com/movie/top100List.nhn?&rankingTypeCode=PC_M&page= 월간

def getPageLinksWantRange(startPageNo, lastPageNo) :
  links = [] #100개의 영화 링크를 수집할 리스트 변수 
  
  for pageNo in range(startPageNo -1, lastPageNo) :
    #실시간 Top 100 영화 링크 수집.
    url = "https://serieson.naver.com/movie/top100List.nhn?&rankingTypeCode=PC_M&page=" + str(pageNo +1)

    req = requests.get(url) #페이지 접속
    soup = BeautifulSoup(req.text, 'lxml') #html 파싱을 위한 객체 생성
    movielinks = soup.select('div.lst_thum_wrap ul li a[href]') #<div class="lst_thum_wrap"> -> <ul> -> <li> -> <a href ="">

    for movielink in movielinks:
      link = str(movielink.get('href'))

      links.append('https://series.naver.com' + link) #접속할 수 있는 전체 링크 형태로 변환해서 더함
  return links

네이버 영화 제목, 평점,장르, 줄거리 크롤링

In [ ]:
def getMovieDataFromNaverSeries(links) :
  title_infos = [] #제목
  content_infos = [] #줄거리
  genre_infos = [] #장르
  score_infos =[] #평점
  data_infos = [] #개봉일 

  url2 = 'https://www.naver.com'

  driver = webdriver.Chrome('chromedriver',options=options)
  driver.get(url2)
  time.sleep(3.0) #3초 휴식

  driver.find_element_by_css_selector('body').send_keys(Keys.CONTROL + 't')

  for link in links:
    print(link + ' 수집 중 ......')
    driver.switch_to.window(driver.window_handles[-1]) #새 창 활성화
    time.sleep(0.1)
    driver.get(link)
    time.sleep(0.1)
    driver.switch_to.window(driver.window_handles[0])
    time.sleep(3.0)

    html_source = driver.page_source # req.text 같은 동작
    html_soup = BeautifulSoup(html_source,'lxml')

    #청불 영화 크롤링 할 경우 -> 인증 -> 수집 제외
    flag = html_soup.text[0:10]

    newflag = "".join(flag)
    newflag = newflag.replace('\n','')

    if newflag == '네이버' :
      time.sleep(3.0)

      #평점 수집
      score = driver.find_element_by_css_selector('div.score_area > em')
      score = float(score.text)
      score = int(score) #정수형으로 변경
      score_infos.append(score)
      print(score)


      #장르 수집
      genre = driver.find_element_by_css_selector('li.info_lst > ul > li:nth-child(4)').get_attribute('textContent')
      genre = genre.replace('장르','')
      genre = genre.replace('\n','')
      genre = genre.replace('\t','')
      genre_infos.append(genre)

      #제목, 줄거리 
      try :
        movieinfoUrl = driver.find_element_by_css_selector('li.info_lst > ul > li:nth-child(7) > a').get_attribute('href')
      except :
        movieinfoUrl = driver.find_element_by_css_selector('li.info_lst > ul > li:nth-child(6) > a').get_attribute('href')

      
      #영화 상세보기 페이지로 이동
      movie_req = requests.get(movieinfoUrl)

      #제목 수집
      movie_soup = BeautifulSoup(movie_req.text, 'lxml')
      title = movie_soup.head.find('meta',{'property' : "og:title"}).get('content')
      title_infos.append(title)

      #줄거리 수집
      contents_texts = movie_soup.select('div.story_area > p.con_tx')

      if len(contents_texts)  == 0 :
        content_infos.append('줄거리 오류')

      else:
        for content in contents_texts:
          #줄거리 데이터 클랜징 작업
          temp = content.text
          temp = temp.replace('\r','')    #줄바꿈 제거
          temp = temp.replace('\xa0','')  #공백 제거
          content_infos.append(temp)
      
    elif newflag == '' :
      print('청불 영화로 데이터 수집하지 않습니다.')

  print('수집을 완료합니다.....')
  print(len(score_infos), len(genre_infos), len(content_infos))
  driver.close()

  movie_dic = {"제목" : title_infos, "평점" : score_infos, "장르" : genre_infos, "줄거리" : content_infos}

  #딕셔너리 -> 데이터프레임
  movie_df = pd.DataFrame(movie_dic)

  #수집된 정보 중에 중복 데이터 삭제
  movie_df2 = movie_df.drop_duplicates("줄거리",keep = 'first') #줄거리 비교해서 중복된 영화를 삭제(첫번째 수집 영화만 남김)

  return movie_df2

CSV 파일로 저장

In [ ]:
def dftoCsv(movie_df, num):
  try:
    movie_df.to_csv(("movie_my_data_"+str(num) + '.csv'),
                 sep=',', na_rep='NaN',encoding='euc-kr') #encoding='utf-8-sig' 로 하면 한글 안깨짐
  except:
    print('Error')

크롤링 실행

In [ ]:
#1) 크롤링 할 링크 수집

links = getPageLinksWantRange(1,5)

#2) 영화 상세 정보 수집
movie_df2 = getMovieDataFromNaverSeries(links)

#3) csv 파일 저장
dftoCsv(movie_df2, len(movie_df2))

https://series.naver.com/movie/detail.nhn?productNo=6119777 수집 중 ......
7
https://series.naver.com/movie/detail.nhn?productNo=6176165 수집 중 ......
8
https://series.naver.com/movie/detail.nhn?productNo=6158879 수집 중 ......
9
https://series.naver.com/movie/detail.nhn?productNo=6098871 수집 중 ......
9
https://series.naver.com/movie/detail.nhn?productNo=6023769 수집 중 ......
9
https://series.naver.com/movie/detail.nhn?productNo=6139377 수집 중 ......
8
https://series.naver.com/movie/detail.nhn?productNo=6139375 수집 중 ......
9
https://series.naver.com/movie/detail.nhn?productNo=6119778 수집 중 ......
7
https://series.naver.com/movie/detail.nhn?productNo=4356726 수집 중 ......
9
https://series.naver.com/movie/detail.nhn?productNo=6176518 수집 중 ......
청불 영화로 데이터 수집하지 않습니다.
https://series.naver.com/movie/detail.nhn?productNo=3293948 수집 중 ......
청불 영화로 데이터 수집하지 않습니다.
https://series.naver.com/movie/detail.nhn?productNo=6039515 수집 중 ......
0
https://series.naver.com/movie/detail.nhn?productNo=6176542 수집 중 ......


크롤링 한 영화 데이터 랭킹 시각화

In [1]:
import plotly.express as px
import plotly.graph_objects as go
import pandas as pd
import numpy as np

In [3]:
df = pd.read_csv('/content/실시간.csv',encoding='euc-kr')
df.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 72 entries, 0 to 71
Data columns (total 5 columns):
 #   Column      Non-Null Count  Dtype 
---  ------      --------------  ----- 
 0   Unnamed: 0  72 non-null     int64 
 1   제목          72 non-null     object
 2   평점          72 non-null     int64 
 3   장르          72 non-null     object
 4   줄거리         72 non-null     object
dtypes: int64(2), object(3)
memory usage: 2.9+ KB


In [4]:
df.head()

,Unnamed: 0,제목,평점,장르,줄거리
0,0,극장판 귀멸의 칼날: 무한열차편,9,애니메이션,혈귀로 변해버린 여동생 ‘네즈코’를 인간으로 되돌릴 단서를 찾아 비밀조직 귀살대에 ...
1,1,콰이어트 플레이스,7,공포/스릴러,소리를 내는 순간 공격받는 극한의 상황 속에서 살아남기 위한 한 가족의 숨막히는 사...
2,2,킬러의 보디가드,8,액션,세상 제일 잘난 맛에 사는 섭외 1순위 앵그리 보디가드가지명수배 1순위 구강 액션 ...
3,3,미나리,7,드라마,"""미나리는 어디서든 잘 자라""낯선 미국, 아칸소로 떠나온 한국 가족.가족들에게 뭔가..."
4,4,소울,9,애니메이션,뉴욕에서 음악 선생님으로 일하던 ‘조’는꿈에 그리던 최고의 밴드와 재즈 클럽에서 연...


In [5]:
fig = px.scatter(df, x= '제목', y = '평점', color='장르')
fig.show()

In [6]:
df_score = df[df.평점 == 9]
df_score.tail()

,Unnamed: 0,제목,평점,장르,줄거리
52,55,영웅본색,9,액션,"한때 암흑가를 주름잡는 보스였으나 손 씻고 새 삶을 시작한 자호(적룡), 경찰의 길..."
53,56,내생애최고의경기,9,드라마,1913년 US 오픈 당시 영국 챔피언 해리 바든을 꺾고 우승한 20세 프랜시스 위...
59,63,쇼크 웨이브 2,9,액션,뛰어난 폭탄 제거 요원 ‘판청펑’(유덕화)은 현장에서 예기치 못한 사고로 왼쪽 다리...
65,73,만델라 이펙트,9,SF/판타지,"해변에서 휴가를 보내던 중,‘브렌던’의 사랑하는 딸 ‘샘’이 파도에 휩쓸려 실종된다..."
70,81,요요현상,9,다큐멘터리,"취미도 요요, 특기도 요요!요요로 한국을 주름잡던 다섯 명의 ‘요요소년’ 대열, 동..."


평점 9점 영화 중 가장 큰 비율을 차지하고 있는 장르는?

In [12]:
fig = px.pie(df_score, values='평점',names='장르' )
fig.show()


일간 top 100 시각화

In [14]:
df2 = pd.read_csv('/content/일별.csv',encoding='euc-kr')
df2.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 61 entries, 0 to 60
Data columns (total 5 columns):
 #   Column      Non-Null Count  Dtype 
---  ------      --------------  ----- 
 0   Unnamed: 0  61 non-null     int64 
 1   제목          61 non-null     object
 2   평점          61 non-null     int64 
 3   장르          61 non-null     object
 4   줄거리         61 non-null     object
dtypes: int64(2), object(3)
memory usage: 2.5+ KB


In [17]:
fig = px.bar(df2, x= '제목',  y= "평점", color ='장르')
fig.show()

In [15]:
fig = px.pie(df2, values='평점',names='장르' )
fig.show()

주간 TOP 100 시각화

In [21]:
df3 = pd.read_csv('/content/주간.csv',encoding='euc-kr')
df3.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 63 entries, 0 to 62
Data columns (total 5 columns):
 #   Column      Non-Null Count  Dtype 
---  ------      --------------  ----- 
 0   Unnamed: 0  63 non-null     int64 
 1   제목          63 non-null     object
 2   평점          63 non-null     int64 
 3   장르          63 non-null     object
 4   줄거리         63 non-null     object
dtypes: int64(2), object(3)
memory usage: 2.6+ KB


In [22]:
fig = px.bar(df3, x= '제목',  y= "평점", color ='장르')
fig.show()

In [23]:
fig = px.pie(df3, values='평점',names='장르' )
fig.show()

월간 TOP 100 시각화

In [24]:
df4 = pd.read_csv('/content/달별.csv',encoding='euc-kr')
df4.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 64 entries, 0 to 63
Data columns (total 5 columns):
 #   Column      Non-Null Count  Dtype 
---  ------      --------------  ----- 
 0   Unnamed: 0  64 non-null     int64 
 1   제목          64 non-null     object
 2   평점          64 non-null     int64 
 3   장르          64 non-null     object
 4   줄거리         64 non-null     object
dtypes: int64(2), object(3)
memory usage: 2.6+ KB


In [25]:
fig = px.bar(df4, x= '제목',  y= "평점", color ='장르')
fig.show()

In [26]:
fig = px.pie(df4, values='평점',names='장르' )
fig.show()